<a href="https://colab.research.google.com/github/aonovoseltseva/llm-syntax/blob/main/lfg_llm_requests.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Установка Groq

In [ ]:
!pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.5/127.5 kB 5.5 MB/s eta 0:00:00


Инициализация клиента

In [ ]:
from groq import Groq
api_key = 'GROQ_API_KEYclient = Groq(api_key=api_key)

In [ ]:
models = client.models.list()
print(models)

ModelListResponse(data=[Model(id='meta-llama/llama-4-scout-17b-16e-instruct', created=1743874824, object='model', owned_by='Meta', active=True, context_window=131072, public_apps=None, max_completion_tokens=8192), Model(id='llama-3.1-8b-instant', created=1693721698, object='model', owned_by='Meta', active=True, context_window=131072, public_apps=None, max_completion_tokens=131072), Model(id='qwen-qwq-32b', created=1741214760, object='model', owned_by='Alibaba Cloud', active=True, context_window=131072, public_apps=None, max_completion_tokens=131072), Model(id='meta-llama/llama-guard-4-12b', created=1746743847, object='model', owned_by='Meta', active=True, context_window=131072, public_apps=None, max_completion_tokens=1024), Model(id='llama3-8b-8192', created=1693721698, object='model', owned_by='Meta', active=True, context_window=8192, public_apps=None, max_completion_tokens=8192), Model(id='allam-2-7b', created=1737672203, object='model', owned_by='SDAIA', active=True, context_window=

Запросы к LLM

an example for annotation is an adoptated (by me..) version of another example from the #Bozhil Hristov. 2023. LFG and Slavic languages. In Mary Dalrymple (ed.), Handbook
of Lexical Functional Grammar, 1753–1804. Berlin: Language Science Press. DOI: 10.
5281/zenodo.10186030

Prompt option1

In [ ]:
core_concepts = client.chat.completions.create(messages=[
                                                            {"role": "system",
                                                            "content": """
You are a linguist working within the Lexical-Functional Grammar (LFG) framework. Your task is to annotate Russian sentences in 4 structured steps. Please follow this process:

Step 1: Annotate Morphological Features
Assign NUM, CASE, GEND, etc., to all relevant words (especially NPs and adjectives).

Step 2: Mark Syntactic Dependencies
Mark agreement between elements (e.g., adjectives agreeing with nouns) as CONCORD.

Step 3: Identify Grammatical Functions
Assign SUBJ, OBJ, etc., as appropriate, based on syntax and semantics.

Step 4: Build Unified Representation
Write an f-structure including:
- PRED with valence
- SUBJ and OBJ slots filled with syntactic (s) representations and morphological info

Example (for sentence: 'те хорошие дети пришли'):

те (s CONCORD NUM) = PL ; (s CONCORD CASE) = NOM
хорошие (s CONCORD NUM) = PL ; (s CONCORD CASE) = NOM
дети (s CONCORD NUM) = PL ; (s CONCORD CASE) = NOM
пришли (f SUBJ INDEX NUM) = PL
f[PRED 'пришли<SUBJ>';
  SUBJ s[PRED 'дети', SPEC 'те', ADJ {[PRED 'хорошие']}, CONCORD [NUM PL], INDEX [NUM PL]]]
"""
                                                            },
                                                            {
                                                                "role": "system",
                                                                 "content": """Here is the glossary to stay alligned with the Lexical-Functional Grammar Theory:
                                                                 NUM	Number: SG, PL
CASE	Case: NOM, ACC, DAT, etc.
GEND	Gender: MASC, FEM, NEUT
PERS	Person: 1, 2, 3
TENSE	Tense: PAST, PRES, etc.
ASPECT	Aspect: PERF, IMPERF, etc.
VOICE	ACTIVE, PASSIVE
CONCORD	Morphological agreement feature for modifiers (e.g., adjectives agreeing with nouns)"""
                                                            },
                                                            {
                                                                "role": "user",
                                                                "content": """Your task is to annotate the following sentence: 'эти зеленые деревья зацвели' """
                                                            },

                                                            {
                                                                "role": "user",
                                                                "content": """Review your parse.  Are the grammatical functions, categories ans others labeled correctly?"""
                                                            },

                                                        ],
                                                        # Форматирование ответа
                                                        #response_format=
                                                        #{
                                                        #    "type": "json_object"
                                                        #},
                                                        model='llama-3.1-8b-instant', # Тип модели
                                                        stream=False,
                                                        max_tokens=500, # Максимальная длина выдачи, желательно поменять под задачу
                                                        temperature=0.7, # Температура
                                                        top_p=0.5 # Топ-P отвечает за разнообразие выдачи
                                                        ).choices[0].message.content

In [ ]:
print (core_concepts)

Let's review the annotation for the sentence 'эти зеленые деревья зацвели'.

1. Annotate grammatical classes:

эти (s concord NUM) = PL ; (s concord CASE) = NOM ; (s concord GEND) = NEUT
зеленые (s concord NUM) = PL ; (s concord CASE) = NOM ; (s concord GEND) = NEUT
деревья (s concord NUM) = PL ; (s concord CASE) = NOM ; (s concord GEND) = NEUT
затцвели (f SUBJ INDEX NUM) = PL

2. Identify syntactic dependencies (concord, agreement):

The adjectives 'зеленые' agree with the noun 'деревья' in number (PL) and case (NOM), and gender (NEUT).

3. Mark grammatical functions:

The sentence has the following grammatical functions:
- SUBJ: зацвели
- OBJ: деревья

4. Relate f-structure and c-structure in a unified representation:

f[PRED ‘затцвели<SUBJ>’;
  SUBJ s[PRED ‘деревья’, SPEC ‘эти’, ADJ {[PRED ‘зеленые’]}, CONCORD [NUM PL, CASE NOM, GEND NEUT], INDEX [NUM PL]]]

However, there are some issues with the annotation:
- The subject 'эти зеленые деревья' is not correctly annotated as a single

Prompt option2: the glossary is full

In [ ]:
core_concepts = client.chat.completions.create(messages=[
                                                            {
                                                              "role": "system",
                                                              "content":
"""You are a linguist working in the Lexical-Functional Grammar (LFG) framework. Your goal is to perform precise syntactic annotation of Russian sentences, following these four steps:\n\
1. Annotate grammatical categories (e.g., NUM, CASE, GEND) for all relevant words.
2. Identify agreement relations (e.g., concord between determiners, adjectives, and nouns).
3. Assign grammatical functions (e.g., SUBJ, OBJ, etc.) and link them to lexical predicates.
4. Represent the unified f-structure based on grammatical roles and their syntactic expressions.

Use the following format for annotation:
<word> (s concord <FEATURE>) = <VALUE>
<verb> (f <FUNCTION> INDEX <FEATURE>) = <VALUE>
f[PRED ‘<predicate<SUBJ, OBJ, ...>>’; SUBJ ..., OBJ ..., ...]"""
                                                            },
                                                              {
                                                              "role": "system",
                                                              "content": """
Here is an example annotation:
Sentence: 'те хорошие дети пришли'
те (s concord NUM) = PL ; (s concord CASE) = NOM
хорошие (s concord NUM) = PL ; (s concord CASE) = NOM
дети (s concord NUM) = PL ; (s concord CASE) = NOM
пришли (f SUBJ INDEX NUM) = PL
f[PRED ‘пришли<SUBJ>’; SUBJ s[PRED ‘дети’, SPEC ‘те’, ADJ {[PRED ‘хорошие’]}, CONCORD [NUM PL], INDEX [NUM PL]]]"""
                                                            },
                                                            {
                                                              "role": "system",
                                                              "content": """Here is the glossary to stay aligned with the Lexical-Functional Grammar (LFG) Theory:
GRAMMATICAL FEATURES
NUM — Number: SG (singular), PL (plural)
CASE — Case: NOM (nominative), ACC (accusative), DAT (dative), GEN (genitive), INS (instrumental), LOC (locative), etc.  \n\
GEND — Gender: MASC (masculine), FEM (feminine), NEUT (neuter)
PERS — Person: 1 (first), 2 (second), 3 (third)
TENSE — Tense: PAST, PRES, FUT
ASPECT — Aspect: PERF (perfective), IMPERF (imperfective)
VOICE — Voice: ACTIVE, PASSIVE
MOOD — Mood: IND (indicative), IMP (imperative), SUBJ (subjunctive)
CONCORD — Agreement features, usually between determiners, adjectives, and nouns (NUM, CASE, GEND)
INDEX — Feature bundle used to track identity and agreement between elements (e.g., between subject and verb)

SUBJ — Subject (core argument)
OBJ — Object (core argument)
IOBJ — Indirect object
OBL — Oblique (non-core argument, typically marked with preposition or case)
COMP — Closed complement (clausal complement with subject)
XCOMP — Open complement (clausal complement without subject, subject is controlled)
ADJUNCT — Modifier (optional, peripheral element)
SPEC — Specifier (e.g., demonstratives or determiners like 'этот', 'тот')
ADJ — Adjective or modifier inside a noun phrase
STRUCTURE MAPPINGS
C-structure — Constituent structure (phrase structure tree: S → NP + VP, etc.)
F-structure — Functional structure: abstract mapping of functions and features (e.g., SUBJ, OBJ, TENSE)
PRED — Predicate representation: 'читать<SUBJ,OBJ>'
(↑ SUBJ) = ↓ — Mapping rule: the daughter node is the SUBJ of the mother node
↑ = ↓ — Identity between nodes (shared features)
Use this glossary to ensure consistency and theoretical accuracy when annotating or revising sentence structures in LFG."""
                                                            },
                                                            {
                                                                "role": "user",
                                                                "content": """Your task is to annotate the following sentence: 'эти зеленые деревья зацвели' """
                                                            },

                                                            {
                                                                "role": "user",
                                                                "content": """Review your parse.  Are the grammatical functions, categories ans others labeled correctly?"""
                                                            },

                                                        ],
                                                        # Форматирование ответа
                                                        #response_format=
                                                        #{
                                                        #    "type": "json_object"
                                                        #},
                                                        model='llama-3.1-8b-instant', # Тип модели
                                                        stream=False,
                                                        max_tokens=500, # Максимальная длина выдачи, желательно поменять под задачу
                                                        temperature=0.7, # Температура
                                                        top_p=0.5 # Топ-P отвечает за разнообразие выдачи
                                                        ).choices[0].message.content

In [ ]:
print (core_concepts)

Prompt option3: the example is given as the last system content (perhaps will be memorised better by the model)


In [ ]:
core_concepts = client.chat.completions.create(messages=[
                                                            {
                                                              "role": "system",
                                                              "content":
"""You are a linguist working in the Lexical-Functional Grammar (LFG) framework. Your goal is to perform precise syntactic annotation of Russian sentences, following these four steps:\n\
1. Annotate grammatical categories (e.g., NUM, CASE, GEND) for all relevant words.
2. Identify agreement relations (e.g., concord between determiners, adjectives, and nouns).
3. Assign grammatical functions (e.g., SUBJ, OBJ, etc.) and link them to lexical predicates.
4. Represent the unified f-structure based on grammatical roles and their syntactic expressions.

Use the following format for annotation:
<word> (s concord <FEATURE>) = <VALUE>
<verb> (f <FUNCTION> INDEX <FEATURE>) = <VALUE>
f[PRED ‘<predicate<SUBJ, OBJ, ...>>’; SUBJ ..., OBJ ..., ...]"""
                                                            },
                                                            {
                                                              "role": "system",
                                                              "content": """Here is the glossary to stay aligned with the Lexical-Functional Grammar (LFG) Theory:
GRAMMATICAL FEATURES
NUM — Number: SG (singular), PL (plural)
CASE — Case: NOM (nominative), ACC (accusative), DAT (dative), GEN (genitive), INS (instrumental), LOC (locative), etc.  \n\
GEND — Gender: MASC (masculine), FEM (feminine), NEUT (neuter)
PERS — Person: 1 (first), 2 (second), 3 (third)
TENSE — Tense: PAST, PRES, FUT
ASPECT — Aspect: PERF (perfective), IMPERF (imperfective)
VOICE — Voice: ACTIVE, PASSIVE
MOOD — Mood: IND (indicative), IMP (imperative), SUBJ (subjunctive)
CONCORD — Agreement features, usually between determiners, adjectives, and nouns (NUM, CASE, GEND)
INDEX — Feature bundle used to track identity and agreement between elements (e.g., between subject and verb)

SUBJ — Subject (core argument)
OBJ — Object (core argument)
IOBJ — Indirect object
OBL — Oblique (non-core argument, typically marked with preposition or case)
COMP — Closed complement (clausal complement with subject)
XCOMP — Open complement (clausal complement without subject, subject is controlled)
ADJUNCT — Modifier (optional, peripheral element)
SPEC — Specifier (e.g., demonstratives or determiners like 'этот', 'тот')
ADJ — Adjective or modifier inside a noun phrase
STRUCTURE MAPPINGS
C-structure — Constituent structure (phrase structure tree: S → NP + VP, etc.)
F-structure — Functional structure: abstract mapping of functions and features (e.g., SUBJ, OBJ, TENSE)
PRED — Predicate representation: 'читать<SUBJ,OBJ>'
(↑ SUBJ) = ↓ — Mapping rule: the daughter node is the SUBJ of the mother node
↑ = ↓ — Identity between nodes (shared features)
Use this glossary to ensure consistency and theoretical accuracy when annotating or revising sentence structures in LFG."""
                                                            },
                                                            {
                                                              "role": "system",
                                                              "content": """
Here is an example annotation:
Sentence: 'те хорошие дети пришли'
те (s concord NUM) = PL ; (s concord CASE) = NOM
хорошие (s concord NUM) = PL ; (s concord CASE) = NOM
дети (s concord NUM) = PL ; (s concord CASE) = NOM
пришли (f SUBJ INDEX NUM) = PL
f[PRED ‘пришли<SUBJ>’; SUBJ s[PRED ‘дети’, SPEC ‘те’, ADJ {[PRED ‘хорошие’]}, CONCORD [NUM PL], INDEX [NUM PL]]]"""
                                                            },
                                                            {
                                                                "role": "user",
                                                                "content": """Your task is to annotate the following sentence: 'эти зеленые деревья зацвели' """
                                                            },

                                                            {
                                                                "role": "user",
                                                                "content": """Review your parse.  Are the grammatical functions, categories ans others labeled correctly?"""
                                                            },

                                                        ],
                                                        # Форматирование ответа
                                                        #response_format=
                                                        #{
                                                        #    "type": "json_object"
                                                        #},
                                                        model='llama-3.1-8b-instant', # Тип модели
                                                        stream=False,
                                                        max_tokens=500, # Максимальная длина выдачи, желательно поменять под задачу
                                                        temperature=0.7, # Температура
                                                        top_p=0.5 # Топ-P отвечает за разнообразие выдачи
                                                        ).choices[0].message.content

In [ ]:
print (core_concepts)

Benchmarks: treebank ru?
https://universaldependencies.org/treebanks/ru_syntagrus/index.html
https://universaldependencies.org/treebanks/ru_gsd/index.html#ud-russian-gsd
https://universaldependencies.org/treebanks/ru_taiga/index.html#ud-russian-taiga
https://universaldependencies.org/treebanks/ru_pud/index.html#ud-russian-pud